# 🧪 T-Poti From Scratch: Ultra-Low Precision (2026)

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/chronology/tpoti_demo.ipynb)

## 📖 The Theory: Binary and Quaternary Frontiers

T-Poti represents the bleeding edge of quantization research for 2026. As models grow, even 4-bit INT4 is sometimes too large for extreme edge devices. T-Poti focuses on **1-bit (Binary)** and **2-bit (Quaternary)** precision without requiring extensive retraining.

### The Difficulty of 1-bit
When you only have two levels (e.g., `-1` and `1`), the "quantization error" is massive. Standard rounding fails completely. T-Poti use a combination of:
1.  **Optimal Scaling**: Finding the scalar $\alpha$ that minimizes $||W - \alpha \cdot sign(W)||^2$. 
2.  **Error Compensation**: Iterative refinement of the binary mask to preserve the most important activation directions.

---

In [ ]:
import torch

def tpoti_1bit_quantize(W):
    """
    Manual implementation of Optimal Binary Quantization (1-bit).
    Maps W -> alpha * sign(W)
    """
    # 1. Optimal Alpha calculation for a binary mask
    # Formally: alpha = mean(abs(W))
    alpha = torch.mean(torch.abs(W))
    
    # 2. Map to Binary {-1, 1}
    B = torch.sign(W)
    B[B == 0] = 1 # Handle zeros
    
    return B, alpha

def tpoti_2bit_quantize(W):
    """
    Manual 2-bit (Quaternary) quantization.
    Levels: {-1, -0.33, 0.33, 1} scaled by alpha.
    """
    # Normalize to [-1, 1]
    abs_max = torch.max(torch.abs(W))
    W_norm = W / abs_max
    
    # 2-bit levels
    levels = torch.tensor([-1.0, -0.33, 0.33, 1.0])
    
    # Map each value to closest level
    W_flat = W_norm.view(-1, 1)
    diff = torch.abs(W_flat - levels.view(1, -1))
    indices = torch.argmin(diff, dim=1)
    
    W_q = levels[indices].view(W.shape)
    return W_q, abs_max

# Demonstration
W = torch.randn(1024, 1024)
B, alpha = tpoti_1bit_quantize(W)
W_target_2b, scale_2b = tpoti_2bit_quantize(W)

print(f"--- 1-bit Result ---")
print(f"Binary Mask Sample: {B[0, :5].tolist()}")
print(f"Optimal Alpha: {alpha:.4f}")
print(f"1-bit MSE: {torch.mean((W - B*alpha)**2):.6f}")

print(f"\n--- 2-bit Result ---")
print(f"Quaternary Sample: {W_target_2b[0, :5].tolist()}")
print(f"2-bit MSE: {torch.mean((W - W_target_2b*scale_2b)**2):.6f}")